In [4]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ain-space-org/wandb-registry-model/gaia calibrated:v0', type='model')
artifact_dir = artifact.download()
artifact_dir

wandb: Currently logged in as: bernddoser to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: WARNING Registries can be linked/fetched using a shorthand form without specifying the organization name. Try using shorthand path format: <my_registry_name>/<artifact_name> or just <my_registry_name> if fetching just the project.
wandb:   1 of 1 files downloaded.  


'/home/doserbd/git/Spherinator/gaia/artifacts/model-qf4roy5i:v0'

In [1]:
# import yaml
# import importlib

# config = yaml.load(open('model.yaml'), Loader=yaml.FullLoader)
# model_class_path = config["model"]["class_path"]
# module_name, class_name = model_class_path.rsplit(".", 1)
# module = importlib.import_module(module_name)
# model_class = getattr(module, class_name)
# model_init_args = config["model"]["init_args"]
# model = model_class(**model_init_args)

In [2]:
import torch
from spherinator.models import (
    ConsecutiveConv1DLayer,
    ConvolutionalEncoder1DGen,
    DenseModel,
    VariationalAutoencoderPure,
)

cnn_layers=[ConsecutiveConv1DLayer(kernel_size=5,
                                   num_layers=1,
                                   base_channel_number=16,
                                   pooling=torch.nn.MaxPool1d(2, ceil_mode=True)),
            ConsecutiveConv1DLayer(kernel_size=7,
                                   num_layers=1,
                                   base_channel_number=32,
                                   pooling=torch.nn.MaxPool1d(2, ceil_mode=True)),
            ConsecutiveConv1DLayer(kernel_size=9,
                                   num_layers=1,
                                   base_channel_number=64,
                                   pooling=torch.nn.MaxPool1d(2, ceil_mode=True))]
encoder = ConvolutionalEncoder1DGen(input_dim=[1, 343],
                                    output_dim=3,
                                    cnn_layers=cnn_layers)
decoder = DenseModel(layer_dims=[3, 16, 64, 256, 343],
                     output_shape=[1, 343])
model = VariationalAutoencoderPure(encoder=encoder,
                                   decoder=decoder,
                                   z_dim=3,
                                   beta=1.0e-3)

input = torch.randn(1, 1, 343)
model(input)

/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.


((tensor([[-0.4091,  0.9021, -0.1373]], grad_fn=<DivBackward0>),
  tensor([[1.6134]], grad_fn=<AddBackward0>)),
 (PowerSpherical(loc: torch.Size([1, 3]), scale: 1.6133633852005005),
  HypersphericalUniform(dim=3, device=cpu, dtype=torch.float32)),
 tensor([[0.2436, 0.8303, 0.5012]], grad_fn=<SubBackward0>),
 tensor([[[-6.8057e-03,  1.9459e-02, -5.3595e-02, -3.7205e-02, -2.3930e-02,
           -5.9241e-02, -1.8189e-02, -5.7850e-02,  6.2767e-02,  1.3388e-03,
           -2.4770e-02, -1.9696e-02, -8.3411e-02,  1.4066e-02,  1.2744e-02,
            1.1135e-01,  6.4969e-02,  3.3827e-02, -3.3790e-02,  5.3837e-02,
            2.1002e-02,  4.9876e-02, -5.6155e-02,  1.1612e-02,  2.3143e-03,
            8.6934e-02, -9.7613e-02, -3.5659e-02,  3.8975e-02,  6.6614e-02,
           -3.3168e-03,  1.0015e-03,  1.0494e-01,  7.9843e-02,  1.2305e-01,
           -3.4582e-02,  2.4456e-03, -6.8078e-02,  1.3040e-02, -2.1108e-02,
           -4.7508e-02,  7.1821e-02,  5.3122e-02, -2.5421e-02, -5.0033e-02,
       

In [5]:
import torch

ckpt = artifact_dir + "/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
checkpoint

{'epoch': 185,
 'global_step': 44082,
 'pytorch-lightning_version': '2.5.0.post0',
 'state_dict': OrderedDict([('encoder.cnn.0.0.0.weight',
               tensor([[[-0.2702, -0.5835, -0.2809, -0.2388, -0.2410]],
               
                       [[ 0.1957,  0.2095, -0.1171,  0.0027, -0.3216]],
               
                       [[-0.1548,  0.4688,  0.3817, -0.3931, -0.1283]],
               
                       [[-0.4041,  0.3574, -0.3008,  0.1009, -0.1252]],
               
                       [[-0.5530,  0.3542,  0.4249, -0.3230, -0.2633]],
               
                       [[-0.1498, -0.3973,  0.0577,  0.4716, -0.1178]],
               
                       [[-0.3342, -0.0057, -0.1332, -0.2695, -0.5383]],
               
                       [[ 0.4483, -0.2075, -0.2363, -0.2981,  0.3196]],
               
                       [[ 0.1340, -0.2998,  0.4229, -0.3330,  0.1517]],
               
                       [[-0.3221, -0.3529, -0.1921, -0.4498, -0.0400

In [6]:
model.load_state_dict(checkpoint["state_dict"])
model.eval()

input = torch.randn(1, 1, 343)
model(input)

((tensor([[0.4591, 0.2237, 0.8598]], grad_fn=<DivBackward0>),
  tensor([[28595.6230]], grad_fn=<AddBackward0>)),
 (PowerSpherical(loc: torch.Size([1, 3]), scale: 28595.623046875),
  HypersphericalUniform(dim=3, device=cpu, dtype=torch.float32)),
 tensor([[0.4589, 0.2186, 0.8612]], grad_fn=<SubBackward0>),
 tensor([[[0.6207, 0.4858, 0.3959, 0.3607, 0.3641, 0.3616, 0.3613, 0.3652,
           0.3752, 0.4023, 0.3976, 0.3729, 0.3686, 0.3563, 0.3445, 0.3453,
           0.3618, 0.3660, 0.3776, 0.3762, 0.3822, 0.3880, 0.4031, 0.4006,
           0.3978, 0.3815, 0.3635, 0.3408, 0.3238, 0.3041, 0.3024, 0.2992,
           0.3080, 0.3232, 0.3221, 0.3169, 0.2967, 0.2794, 0.2766, 0.2874,
           0.2904, 0.2997, 0.3000, 0.2941, 0.2813, 0.2687, 0.2594, 0.2528,
           0.2441, 0.2426, 0.2381, 0.2463, 0.2625, 0.2752, 0.2848, 0.2973,
           0.3009, 0.3041, 0.3037, 0.2983, 0.3063, 0.3085, 0.3073, 0.3094,
           0.3094, 0.3090, 0.3047, 0.2930, 0.2887, 0.2780, 0.2765, 0.2668,
           0.2655,

In [9]:
import os
os.makedirs("gaia-calibrated-v0", exist_ok=True)

export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx = torch.onnx.dynamo_export(model.variational_encoder, torch.randn(1, 1, 343, device="cpu"), export_options=export_options)
onnx.save("gaia-calibrated-v0/encoder.onnx")

onnx = torch.onnx.dynamo_export(model.decoder, torch.randn(1, 3, device="cpu"), export_options=export_options)
onnx.save("gaia-calibrated-v0/decoder.onnx")

/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/torch/onnx/_internal/_exporter_legacy.py:101: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/torch/onnx/_internal/fx/passes/readability.py:52: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  new_node = self.module.graph.get_attr(normalized_name)
/home/doserbd/.cache/pypoetry/virtualenvs/spherinator-zzYPp9oG-py3.12/lib/python3.12/site-packages/torch/fx/graph.py:1801: UserWarning: Node encoder_cnn_0_0_1_running_mean targe

Applied 2 of general pattern rewrite rules.


In [12]:
!rsync gaia-calibrated-v0 space:/var/www/html/space/models/

Authorized uses only. All activity may be monitored and reported.
skipping directory gaia-calibrated-v0
